# Installation

In [ ]:
!pip install rouge-score

In [ ]:
!pip install evaluate==0.4.0

In [ ]:
!pip install num2words

In [ ]:
import os
os.environ['HF_TOKEN'] = "" #change to your token

# Train model TorchTune

In [ ]:
# Install stable PyTorch, torchvision, torchao stable releases
!pip install torch torchvision torchao
!pip install torchtune

In [ ]:
!pip install triton

In [ ]:
# Check https://github.com/pytorch/torchtune --> select the following model's recipes

# Install Mistral command - hf-token is HuggingFace token
!tune download mistralai/Mistral-7B-Instruct-v0.1 --output-dir tmp/Mistral-7B-v0.1 --hf-token ...

Ignoring files matching the following patterns: *.safetensors
Fetching 13 files:   0% 0/13 [00:00<?, ?it/s]
config.json: 100% 571/571 [00:00<00:00, 3.48MB/s]

generation_config.json: 100% 116/116 [00:00<00:00, 1.12MB/s]

.gitattributes: 100% 1.52k/1.52k [00:00<00:00, 14.8MB/s]
Fetching 13 files:   8% 1/13 [00:00<00:02,  5.22it/s]
pytorch_model.bin.index.json:   0% 0.00/23.9k [00:00<?, ?B/s]

README.md: 100% 5.63k/5.63k [00:00<00:00, 31.0MB/s]
pytorch_model.bin.index.json: 100% 23.9k/23.9k [00:00<00:00, 17.5MB/s]

model.safetensors.index.json: 100% 25.1k/25.1k [00:00<00:00, 19.5MB/s]

pytorch_model-00002-of-00002.bin:   0% 0.00/5.06G [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0% 0.00/9.94G [00:00<?, ?B/s]


special_tokens_map.json: 100% 414/414 [00:00<00:00, 4.18MB/s]



tokenizer.json:   0% 0.00/1.80M [00:00<?, ?B/s]



tokenizer.model:   0% 0.00/493k [00:00<?, ?B/s]




tokenizer_config.json: 100% 2.10k/2.10k [00:00<00:00, 20.5MB/s]
tokenizer.model: 100% 493k/493k [00:00<00:

In [ ]:
# select and configurate mistral recipe according to what you want. We provide a sample config for customized dataset in our Repo
!tune run lora_finetune_single_device --config mistral_lora.yaml

Running LoRAFinetuneRecipeSingleDevice with resolved config:

batch_size: 8
checkpointer:
  _component_: torchtune.training.FullModelHFCheckpointer
  checkpoint_dir: tmp/Mistral-7B-v0.1
  checkpoint_files:
  - pytorch_model-00001-of-00002.bin
  - pytorch_model-00002-of-00002.bin
  model_type: MISTRAL
  output_dir: tmp/torchtune/mistral_7B/lora_single_device
  recipe_checkpoint: null
compile: false
dataset:
  _component_: torchtune.datasets.instruct_dataset
  data_files: train.json
  packed: true
  source: json
  split: train
  train_on_input: false
device: cuda
dtype: bf16
enable_activation_checkpointing: true
enable_activation_offloading: false
epochs: 5
gradient_accumulation_steps: 16
log_every_n_steps: 1
log_peak_memory_stats: true
loss:
  _component_: torchtune.modules.loss.CEWithChunkedOutputLoss
lr_scheduler:
  _component_: torchtune.training.lr_schedulers.get_cosine_schedule_with_warmup
  num_warmup_steps: 100
max_steps_per_epoch: null
metric_logger:
  _component_: torchtune.tra

# Base

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset

model_name = "mistralai/Mistral-7B-Instruct-v0.1"
model = AutoModelForCausalLM.from_pretrained(model_name)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

tokenizer = AutoTokenizer.from_pretrained(model_name)

model.eval()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): MistralRMSNorm((4096,), eps=1e-05)
     

In [ ]:
# 13GB

In [ ]:
import re
from num2words import num2words

#as the dataset is translated using GPT 4o mini, we only need to remove special characters in the text to make sure
def remove_special_characters(text):
    """
    More careful handling of quotation marks and numbers
    """
    text = re.sub(r'\(([^)]+)\)', r' \1 ', text)

    text = re.sub(r'http\S+|www\S+|https\S+', '', text)
    text = re.sub(r'\S+@\S+', '', text)

    text = re.sub(r'\d+', lambda m: num2words(int(m.group())), text)

    text = re.sub(r'[^a-zA-Z0-9\s.,!?"\'-]', ' ', text)
    return ' '.join(text.split())

In [ ]:
import random

# we will provide our test and train.json in our github repo
data_path = 'test.json'
import json

with open(data_path) as file:
    data = json.load(file)

random.shuffle(data)

inputs = [remove_special_characters(text['input']) for text in data]
outputs = [remove_special_characters(text['output']) for text in data]

print(inputs)
print(outputs)

["Initial Diagnosis. Based on the patient's symptoms and existing examination results, a preliminary diagnosis of obstructive jaundice due to a potential biliary obstruction, possibly caused by a gallbladder tumor or a pancreatic tumor, can be made. The rationale behind this diagnosis is as follows one. Symptoms The patient has been experiencing discomfort in the upper abdomen and weight loss for more than three months, along with the onset of jaundice that has progressively worsened over the past month accompanied by low-grade fever. These symptoms suggest a pathological process in the hepatobiliary system. two. Physical Examination The presence of jaundice indicates elevated bilirubin levels, which is often caused by obstruction of the bile ducts. The palpable enlarged gallbladder suggests acute cholecystitis or a more chronic obstruction leading to gallbladder distension. three. Possible Causes Given the age of the patient and the combination of symptoms, potential underlying causes

In [ ]:
def predict_text(text):
  input = tokenizer(text, max_length=512, truncation=True, return_tensors="pt")
  input = input.to('cuda')

  result_ids = model.generate(
      input_ids=input["input_ids"],
      attention_mask=input["attention_mask"],
      max_length = 512,
      temperature=0.7,
      pad_token_id=tokenizer.pad_token_id,
      eos_token_id=tokenizer.eos_token_id,
      do_sample=True,
      top_p=0.9
  )

  result = tokenizer.decode(result_ids[0], skip_special_tokens=True)
  return result

In [ ]:
gen_text = [predict_text(input) for input in inputs]

In [ ]:
print("Input: ", inputs[0])
print("-"*30)
print("Generated output: ", gen_text[0])
print("-"*30)
print("Expected output: ", outputs[0])

Input:  Initial Diagnosis. Based on the patient's symptoms and existing examination results, a preliminary diagnosis of obstructive jaundice due to a potential biliary obstruction, possibly caused by a gallbladder tumor or a pancreatic tumor, can be made. The rationale behind this diagnosis is as follows one. Symptoms The patient has been experiencing discomfort in the upper abdomen and weight loss for more than three months, along with the onset of jaundice that has progressively worsened over the past month accompanied by low-grade fever. These symptoms suggest a pathological process in the hepatobiliary system. two. Physical Examination The presence of jaundice indicates elevated bilirubin levels, which is often caused by obstruction of the bile ducts. The palpable enlarged gallbladder suggests acute cholecystitis or a more chronic obstruction leading to gallbladder distension. three. Possible Causes Given the age of the patient and the combination of symptoms, potential underlying 

In [ ]:
import evaluate
rouge = evaluate.load('rouge')

In [ ]:

#Compute relevant rouge score
score = rouge.compute(
    predictions=gen_text,
    references=outputs,
    use_aggregator=True,
    use_stemmer=True,
)

print("Original score: ", score)

# Trained

In [ ]:
import torch
torch.cuda.empty_cache()
torch.cuda.reset_peak_memory_stats()
torch.cuda.empty_cache()

In [ ]:
del model #free cuda resources

In [ ]:
# del model
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "mistralai/Mistral-7B-Instruct-v0.1"
model = AutoModelForCausalLM.from_pretrained(model_name)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# Fully trained
combined_state_dict = {}
state_dict_paths = [
    '/tmp/torchtune/mistral_7B/lora_single_device/hf_model_0001_4.pt', #change to your saved locations
    '/tmp/torchtune/mistral_7B/lora_single_device/hf_model_0002_4.pt'
]

for state_dict_path in state_dict_paths:
    state_dict = torch.load(state_dict_path)
    combined_state_dict.update(state_dict)

# Load the combined state dict into the model
model.load_state_dict(combined_state_dict, strict=False)
model = model.to('cuda')
model.eval()

<ipython-input-20-288711f9227a>:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(state_dict_path)


MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): MistralRMSNorm((4096,), eps=1e-05)
     

In [ ]:
gen_text = [predict_text(input) for input in inputs]

In [ ]:
print("Input: ", inputs[0])
print("-"*30)
print("Generated output: ", gen_text[0])
print("-"*30)
print("Expected output: ", outputs[0])
# gen_text[0]

Input:  Initial Diagnosis. Based on the patient's symptoms and existing examination results, a preliminary diagnosis of obstructive jaundice due to a potential biliary obstruction, possibly caused by a gallbladder tumor or a pancreatic tumor, can be made. The rationale behind this diagnosis is as follows one. Symptoms The patient has been experiencing discomfort in the upper abdomen and weight loss for more than three months, along with the onset of jaundice that has progressively worsened over the past month accompanied by low-grade fever. These symptoms suggest a pathological process in the hepatobiliary system. two. Physical Examination The presence of jaundice indicates elevated bilirubin levels, which is often caused by obstruction of the bile ducts. The palpable enlarged gallbladder suggests acute cholecystitis or a more chronic obstruction leading to gallbladder distension. three. Possible Causes Given the age of the patient and the combination of symptoms, potential underlying 

In [ ]:
score = rouge.compute(
    predictions=gen_text,
    references=outputs,
    use_aggregator=True,
    use_stemmer=True,
)

print("Trained score: ", score)

# LORA

In [ ]:
del model
model = AutoModelForCausalLM.from_pretrained(model_name)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)


tokenizer = AutoTokenizer.from_pretrained(model_name)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
## LORA
from peft import PeftModel

adapter_path = "/content/tmp/torchtune/mistral_7B/lora_single_device"
model = PeftModel.from_pretrained(model, adapter_path)


In [ ]:
# model = model.merge_and_unload()
model.eval()

PeftModel(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32000, 4096)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralSdpaAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=64, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=64, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): Linear(in_features=4096, out_fe

In [ ]:
gen_text = [predict_text(input) for input in inputs]

In [ ]:
print("Input: ", inputs[0])
print("-"*30)
print("Generated output: ", gen_text[0])
print("-"*30)
print("Expected output: ", outputs[0])
# gen_text[0]

Input:  Initial Diagnosis. Based on the patient's symptoms and existing examination results, a preliminary diagnosis of obstructive jaundice due to a potential biliary obstruction, possibly caused by a gallbladder tumor or a pancreatic tumor, can be made. The rationale behind this diagnosis is as follows one. Symptoms The patient has been experiencing discomfort in the upper abdomen and weight loss for more than three months, along with the onset of jaundice that has progressively worsened over the past month accompanied by low-grade fever. These symptoms suggest a pathological process in the hepatobiliary system. two. Physical Examination The presence of jaundice indicates elevated bilirubin levels, which is often caused by obstruction of the bile ducts. The palpable enlarged gallbladder suggests acute cholecystitis or a more chronic obstruction leading to gallbladder distension. three. Possible Causes Given the age of the patient and the combination of symptoms, potential underlying 

In [ ]:
score = rouge.compute(
    predictions=gen_text,
    references=outputs,
    use_aggregator=True,
    use_stemmer=True,
)

print("Lora score: ", score)